In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_rel

In [ ]:
seeds = [0, 1, 2, 3, 4]


In [ ]:
from scipy.stats import ttest_rel
import os, sys
sys.path.append(os.path.abspath(".."))

In [ ]:
from src.preprocessing import (
    load_penguins_data,
    load_thyroid_data,
    encode_categorical_features,
    train_test_split_stratified
)
from src.models import id3_train, id3_predict
from src.Metrics import compute_metrics

from sklearn.neighbors import KNeighborsClassifier


In [ ]:
X, y = load_thyroid_data()
X = encode_categorical_features(X)

In [9]:
seeds = [0, 1, 2, 3, 4]

f1_id3 = []
f1_knn = []

for seed in seeds:
    X_train, X_test, y_train, y_test = train_test_split_stratified(
        X, y, test_size=0.2, random_state=seed
    )

    # ----- ID3 -----
    tree = id3_train(X_train, y_train)
    y_pred_id3 = id3_predict(tree, X_test)
    _, _, _, f1 = compute_metrics(y_test.values, y_pred_id3)
    f1_id3.append(f1)

    # ----- k-NN -----
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(X_train, y_train)
    y_pred_knn = knn.predict(X_test)
    _, _, _, f1 = compute_metrics(y_test.values, y_pred_knn)
    f1_knn.append(f1)


In [ ]:
results = pd.DataFrame({
    "Seed": seeds,
    "F1_ID3": f1_id3,
    "F1_kNN": f1_knn
})

results

,Seed,F1_ID3,F1_kNN
0,0,0.869565,0.809524
1,1,0.916667,0.685714
2,2,0.904762,0.705882
3,3,0.930233,0.666667
4,4,0.977778,0.685714


In [ ]:
t_stat, p_value = ttest_rel(f1_id3, f1_knn)

print("Paired t-test")
print(f"t-statistic = {t_stat:.4f}")
print(f"p-value     = {p_value:.4f}")

Paired t-test
t-statistic = 5.1752
p-value     = 0.0066
